<a href="https://colab.research.google.com/github/AnilZen/centpy/blob/master/notebooks/Scalar_2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quasilinear scalar equation with CentPy in 2d

### Import packages

In [1]:
# Install the centpy package
!pip install centpy


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import numpy and centpy for the solution 
from numpy import pi, sin, cos, abs, min, max
import centpy

In [3]:
# Imports functions from matplotlib and setup for the animation
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

## Equation

We solve the nonlinear scalar conservation law 

\begin{equation}
\partial_t u + \partial_x \sin u + \frac{1}{3} \partial_y u^3= 0,
\end{equation}

on the domain $(x,y,t)\in([0,2\pi]\times[0,2\pi]\times[0,6])$ with initial data

\begin{equation}
u(x,y,0) = \sin \left(x+\frac{1}{2}\right) \cos(2x+y)
\end{equation}

and periodic boundary conditions. The solution is computed using a 144 $\times$ 144 mesh and CFL number 0.9.

In [4]:
pars = centpy.Pars2d(
    x_init=0, x_final=2*pi,
    y_init=0.0, y_final=2*pi,
    J=144, K=144,
    t_final=6.0,
    dt_out=0.1,
    cfl=0.9,
    scheme="sd3",
)

In [5]:
class Scalar2d(centpy.Equation2d):
    def initial_data(self):
        x = self.xx.T; y = self.yy.T
        return sin(x + 0.5) * cos(2*x + y)

    def boundary_conditions(self, u):
        # x-boundary
        u[0] = u[-4]
        u[1] = u[-3]
        u[-2] = u[2]
        u[-1] = u[3]
        # y-boundary
        u[:, 0] = u[:, -4]
        u[:, 1] = u[:, -3]
        u[:, -2] = u[:, 2]
        u[:, -1] = u[:, 3]

    def flux_x(self, u):
        return sin(u)

    def flux_y(self, u):
        return 1./3 *u**3

    def spectral_radius_x(self, u):
        return abs(cos(u))

    def spectral_radius_y(self, u):
        return u**2

## Solution

In [ ]:
eqn = Scalar2d(pars)
soln = centpy.Solver2d(eqn)
soln.solve()

## Animation

In [ ]:
# Animation
j0 = slice(2, -2)

fig = plt.figure()
ax = plt.axes(xlim=(soln.x_init,soln.x_final), ylim=(soln.y_init, soln.y_final))

ax.set_title("Nonlinear scalar")
ax.set_xlabel("x")
ax.set_ylabel("y")

contours=ax.contour(soln.x[j0], soln.y[j0], soln.u_n[0,j0,j0], 8, colors='black') 
img=ax.imshow(soln.u_n[0,j0,j0], extent=[0, 6.3, 0, 6.3], origin='lower',
           cmap='ocean', alpha=0.5)

fig.colorbar(img)
def animate(i):
    for c in ax.collections:
        c.remove()
    ax.contour(soln.x[j0], soln.y[j0], soln.u_n[i,j0,j0], 8, colors='black') 
    img.set_array(soln.u_n[i,j0,j0])
    img.autoscale()
    
plt.close()
anim = animation.FuncAnimation(fig, animate, frames=soln.Nt, interval=100, blit=False);
HTML(anim.to_html5_video())